In [1]:
import cv2
import matplotlib.pyplot as plt
import time
import os
import numpy as np
%matplotlib inline

In [2]:
subjects = ["", "Makakole", "Hlobisile"]

In [3]:
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('../data/haarcascade_frontalface_alt.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    if (len(faces) == 0):
        return None, None
    (x, y, w, h) = faces[0]
    return gray[y:y+w, x:x+h], faces[0]

In [4]:
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    for dir_name in dirs:
        if not dir_name.startswith("p"):
            continue;
            
        label = int(dir_name.replace("p", ""))
        subject_dir_path = data_folder_path + "/" + dir_name
        subject_images_names = os.listdir(subject_dir_path)
        
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue;
            image_path = subject_dir_path + "/" + image_name
            image = cv2.imread(image_path)
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            face, rect = detect_face(image)
            if face is not None:
                faces.append(face)
                labels.append(label)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [5]:
print("Preparing data...")
faces, labels = prepare_training_data("../data/training")
print("Data prepared")
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  31
Total labels:  31


In [6]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

#face_recognizer = cv2.face.EigenFaceRecognizer_create()

#face_recognizer = cv2.face.FisherFaceRecognizer_create()

In [7]:
face_recognizer.train(faces, np.array(labels))

In [8]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [9]:
def predict(test_img):
    img = test_img.copy()
    face, rect = detect_face(img) 
    label= face_recognizer.predict(face)
    print('label ', label[0])
    label_text = subjects[label[0]]
    draw_rectangle(img, rect)
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img

In [ ]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread("../data/test/test1.jpg")
test_img2 = cv2.imread("../data/test/test2.jpg")
test_img3 = cv2.imread("../data/test/test3.jpg")

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
predicted_img3 = predict(test_img3)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], predicted_img1)
cv2.imshow(subjects[2], predicted_img2)
cv2.imshow(subjects[2], predicted_img3)
cv2.waitKey(0)
cv2.destroyAllWindows()

Predicting images...
label  2
label  1
label  2
Prediction complete
